# Lecture 4 - Latent Variable Models

# Overview

TL;DR:


We want to be able to model a rich space $x$ (e.g. images of bedrooms) using a **latent** represantion $z \sim p_Z(z)$ (naive: how many beds, lamps, windows, etc) that we pick:

<img src="https://www.evernote.com/l/AguDzvDhelxOXZxCuQdAEcuxqmJ47bQU_LwB/image.png" alt="drawing" width="150"/>


* **VLB/ELBO** can be used as a more manageable objective. 

## Sample

$$ z \sim p_Z(z) $$
$$ x \sim p_\theta(x | z) $$
## Evaluate likelihood

$$ p_\theta(x) = \sum_z p_Z(z)p_\theta(x | z) $$

## Train

We use MLE as usual. Objective is:

$$\max_\theta\sum_i \log p_\theta(x^{(i)}) = 
\max_\theta \sum_i \log \sum_z p_Z(z)p_\theta(x^{(i)} | z)$$
## Representation 

$$ x \rightarrow z $$

If $z$ is the compact representation of our data, we usually we want to be able to go in the opposite direction. We should be able to do this using Baye's rule:

$$ p_\theta(z | x) = \frac{p_\theta(x | z) p_Z(z)}{p_\theta(x) }$$

Although, it is not always tractable to do this.
## Notes

* Difference from Flow Models: Flow models assume that x and z have the same dimensionality, so that the flow is a biyection.
## It's all about how big the $Z$ space is

If the $Z$ space is small enough, say, when $z$ takes a small number of discrete values. E.g. 

$$z \in {A, B, C}  \\
p_Z(z = A) = p_Z(z = B) = p_Z(z = C) = \frac13  \\
p_\theta(x | z = k) = \frac{1}{(2\pi)^{\frac{n}{2}}\left|\Sigma_k\right|^{\frac12}}\exp\left(-\frac12(x - \mu_k)^\top\Sigma_k^{-1}(x - \mu_k)\right)
$$

### Training objective is 

$$ \max_\theta\sum_i\log p_\theta(x^{(i)}) \\
= \max_{\mu, \Sigma}\sum_i \log\left[ 
\frac13 \frac{1}{(2\pi)^{\frac{n}{2}}\left|\Sigma_A\right|^{\frac12}}\exp\left(-\frac12(x - \mu_A)^\top\Sigma_A^{-1}(x - \mu_A)\right) + \\ 
\frac13 \frac{1}{(2\pi)^{\frac{n}{2}}\left|\Sigma_B\right|^{\frac12}}\exp\left(-\frac12(x - \mu_B)^\top\Sigma_B^{-1}(x - \mu_B)\right) + \\ 
\frac13 \frac{1}{(2\pi)^{\frac{n}{2}}\left|\Sigma_C\right|^{\frac12}}\exp\left(-\frac12(x - \mu_C)^\top\Sigma_C^{-1}(x - \mu_C)\right) \right]$$
This objective can be evaluated for each training point, and we can feed this to an optimizer (SGD, etc)

## What happens when the $Z$ space is big? 

Then the sum

$$p(x^{(i)}) = \sum_z p_Z(z)p_\theta(x^{(i)} | z)$$

Cannot be evaluated exactly. Instead we approximate it

$$\sum_z p_Z(z)p_\theta(x^{(i)} | z) = 
\mathop{\mathbb{E}}_{z \sim Z} p_\theta(x^{(i)} | z)\\\approx 
\frac1K \sum_{k=1}^{K}p_\theta(x^{(i)} | z_k^{(i)})$$
Our objective becomes

$$ \max_\theta \sum_i \log \frac1K \sum_{k=1}^{K}p_\theta(x^{(i)} | z_k^{(i)}) $$

So for every $x^{(i)}$ we'll be sampling $K$ $z$'s
## [Prior Sampling Challenge](https://youtu.be/FMuvUZXMzKM)

Let's say our data $x$ is in N gaussian clusters, and we're fitting a mixture of gaussians, where $z$ determines which gaussian we sample. 

Sampling $z$ from our mixture uniformly results in only $\frac1N$ of the terms (i.e. $p_\theta(x | z)$) being considerable contributions, the others are very close to 0 (as $z$ and $x^{(i)}$ are in different modes, and their gradients are very close to 0. 

<img src="https://www.evernote.com/l/AgsJsb70fnRFKI_hYjcsbLJDX9cfvlD-9EsB/image.png" alt="drawing" width="500"/>

In high dimensions it becomes near impossible to get "lucky" enough that the sampled $z$ is a good match for the training point $x$

# Importance Sampling

Want to compute 

$$\mathbb{E}_{z \sim p_Z(z)}f(z)$$
But (1) hard to sample from $p_Z(z)$, and/or samples from $p_Z(z)$ are not very informative (e.g. the example of N gaussian clusters)
## Main idea

$$\mathbb{E}_{z \sim p_Z(z)}f(z) = 
\sum_z p_Z(z) f(z) \\
= \sum_z \frac{q(z)}{q(z)}p_Z(z)f(z) \\
= \mathbb{E}_{z \sim q(z)} \left[\frac{p_z(z)}{q(z)}f(z) \right] \\
\approx \frac1K\sum_{k=1}^K \frac{p_Z(z^{(k)})}{q(z^{(k)})}f(z^{(k)})$$

With $z^{(k)} \sim q(z)$. We can now sample from $q$ to compute expectation w.r.t $p$
### Importance Sampled Objective

$$\sum_i \log \sum_z p_Z(z)p_\theta(x^{(i)} | z) 
\approx \sum_i \log \frac1K\sum_{k=1}^K \frac{p_Z(z^{(i)}_k)}{q(z^{(i)}_k)}p_\theta(x^{(i)} | z_k^{(i)}) $$

With $z_k^{(i)} \sim q(z_k^{(i)})$
## How to pick $q(z)$ ?

**Need to find distribution $q(z)$ that tells us which $z$ are likely given our data $x^{(i)}$**. This is like learning the reverse mapping $x^{(i)} \rightarrow z$

How about 

$$ q(z) = p_\theta(z | x^{(i)}) = \frac{p(x^{(i)} | z)p_Z(z)}{p_\theta(x^{(i)})} $$

**Issue:** Unclear how to sample from this. From our assumption it is not tractable to compute the denominator (marginalizing over all $z$s is hard)

## Variational Approach

Propose a parameterized dist $q$ that is easy to work with, and learn parameters $\phi$ that approximates it.

### In our case 

E.g. find $q(z) = \mathcal{N}(z; \mu, \sigma^2)$      as close as possible to $p(z | x^{(i)})$

### How?

Minimize $\mathrm{KL}$ divergence:

$$\min_{q(z)} \mathrm{D_{KL}}(q(z) \parallel p(z | x^{(i)})) $$

$$ = \min_{q(z)} \mathop{\mathbb{E}}_{z \sim q(z)} \log \left( \frac{q(z)}{ p(z | x^{(i)})}\right) $$
$$ = \min_{q(z)} \mathop{\mathbb{E}}_{z \sim q(z)}\left[ \log {q(z)} - \log p(z | x^{(i)}) \right]$$

$$ = \min_{q(z)} \mathop{\mathbb{E}}_{z \sim q(z)}\left[ \log {q(z)} - \log \left(\frac{p_\theta(x^{(i)} | z) p_Z(z)}{p_\theta(x^{(i)})} \right) \right]$$
$$ = \min_{q(z)} \mathop{\mathbb{E}}_{z \sim q(z)}\left[ \log {q(z)} + \log p_\theta(x^{(i)}) - \log p_\theta(x^{(i)}| z) - \log p_Z(z)\right]$$
$$ = \min_{q(z)} \mathop{\mathbb{E}}_{z \sim q(z)}\left[ \log {q(z)} - \log p_\theta(x^{(i)}| z) - \log p_Z(z)\right]$$

All the quantities in this objective are readily computable:

* $q(z)$ is easy to compute by choice
* $p_Z(z)$ is also easy to compute by design
* $p_\theta(x^{(i)} | z)$ is a neural network, and even though $\theta$ may not be the final one, we can still compute it efficiently

**Drawback**: We need to learn a $q$ separately fo **each** $x^{(i)}$

## [Amortized Inference](https://www.youtube.com/watch?v=FMuvUZXMzKM&t=2092s)

**We can do better!**

<img src="https://www.evernote.com/l/Ags_LC-RXlhPXoxXd-2N7mFqhanpQrE50w4B/image.png" alt="drawing" width="300"/>

### Old objective

$$\min_{q(z)} \mathrm{D_{KL}}(q(z) \parallel p(z | x^{(i)})) $$

This was being done per datapoint $x^{(i)}$

### Amortized formulation

Add all the things! Let a NN learn the reverse mapping $x \rightarrow z$

#### Amortized obective:

$$\min_\phi \sum_i \mathrm{D_{KL}}\left(q_\phi(z | x^{(i)}) \parallel p(z | x^{(i)}\right)$$

The old formulation required an optimization step per datapoint $x^{(i)}$ to find its corresponding $q$. With the new objective, we train a network $q_\phi$ to learn this.

**Tradeoffs**

* +Faster
* +Regularized
* -Not as precise

### Example

We have an image $x$, and $q_\phi(z | x) = \mathcal{N}(\mu_\phi(x), \sigma^2(x))$. Passing $x$ through $q$, returns a $\mu$ and $\sigma$ which tells us the distribution of the $z$ that $x$ was sampled from.
# [Importance Weighted AutoEncoder (IWAE)](https://youtu.be/FMuvUZXMzKM?t=2815)

Combine all of the above:

$$\mathcal{L}_K = \left[ \sum_i \log \frac1K\sum_{k=1}^K \frac{p_Z(z^{(i)}_k)}{q_\phi(z^{(i)}_k)}p_\theta(x^{(i)} | z_k^{(i)})  - \sum_i \mathrm{KL}(q_\phi(z | x^{(i)}) \parallel p(z | x^{(i)})\right]$$

$$ \max_{\theta, \phi} \mathcal{L}_K $$

With $z_k^{(i)} \sim q_\phi(z_k^{(i)})$ 


**Theorem**

$$\log p(\mathbf{x}) \geq \mathcal{L}_{K+1} \geq \mathcal{L}_K $$

And 

$$\log p(\mathbf{x}) = \lim_{k\rightarrow \infty} \mathcal{L}_K $$

# [VLB / ELBO](https://youtu.be/FMuvUZXMzKM?list=PLwRJQ4m4UJjPiJP3691u-qWwPGVKzSlNP)

$$\max_\theta\sum_i\log p_\theta(x^{(i)}) $$

$$ = \max_\theta\sum_i\log\left(\sum_z \frac{q(z)}{q(z)} p_z(z) p_\theta(x^{(i)} | z) \right) $$

$$ =\max_\theta\sum_i \log \left(\mathop{\mathrm{E}}_{z \sim q(z)} \left[\frac{p_z(z)}{q(z)} p_\theta(x^{(i)} | z)\right]\right) $$

By Jensen's:

$$ \geq\max_\theta\sum_i \mathop{\mathrm{E}}_{z \sim q(z)} \left[ \log \left(\frac{p_z(z)}{q(z)} p_\theta(x^{(i)} | z)\right)\right] $$

$$ =\max_\theta\sum_i \mathop{\mathrm{E}}_{z \sim q(z)} \left[ \log(p_z(z)) + \log(p_\theta(x^{(i)} | z)) - \log(q(z))\right] $$
Equality is met when $ q(z) \propto p_z(z)p_\theta(x^{(i)} | z) \propto p_\theta(z | x^{(i)})$
## [Alternative derivation](https://youtu.be/FMuvUZXMzKM?list=PLwRJQ4m4UJjPiJP3691u-qWwPGVKzSlNP&t=3969)
$$D_{\mathrm{KL}} = \mathop{\mathrm{E}}_{z \sim q(z)} \left[\log(q(z)) - \log(p(z|x))\right]$$
$$ \mathop{\mathrm{E}}_{z \sim q(z)} \left[\log(q(z)) - 
\log(p(z, x)) + \log(p(x))\right] $$
$$ \mathop{\mathrm{E}}_{z \sim q(z)} \left[\log(q(z)) - 
\log(p(z, x)) \right] + \log(p(x)) $$
$$ \mathop{\mathrm{E}}_{z \sim q(z)} \left[\log(q(z)) - 
\log(p(x | z)) - \log(p(z)) \right] + \log(p(x)) $$
$$\Rightarrow D_{\mathrm{KL}}(q(z) \parallel p(z|x)) = \mathop{\mathrm{E}}_{z \sim q(z)} \left[\log(q(z)) - 
\log(p(x | z)) - \log(p(z)) \right] + \log(p(x)) $$
Or
$$\log(p(x)) = 
\mathop{\mathrm{E}}_{z \sim q(z)} \left[
\log(p(x | z)) + \log(p(z)) - \log(q(z)) \right] 
+ D_{\mathrm{KL}}(q(z) \parallel p(z|x)) $$
Therefore the VLB holds, and we have equality when $q(z) \propto p(z | x)$
# [How to optimize an expectation?](https://youtu.be/FMuvUZXMzKM?list=PLwRJQ4m4UJjPiJP3691u-qWwPGVKzSlNP&t=4109)
Given something of the form

$$\max_\phi\mathrm{E}_{z \sim q_\phi(z))}\left[f(z)\right]$$

### Naive approach

Just sample $z^{(i)} \sim q(z)$. However, then when we go to evaluate the gradient we have:

$$\nabla_\phi\left[\frac1K\sum_k f(z^{(k)})\right] = 0 ??? \quad\text{no deps on}~ \phi$$

This is not true...
## Likelihood Ratio Gradient

Expand expectation:

$$\nabla_\phi \left(\mathrm{E}_{z \sim q_\phi(z)}[f(z)]\right) 
= \sum_z \nabla_\phi q_\phi(z) f(z) \\
= \sum_z \frac{q_\phi(z)}{q_\phi(z)}\nabla_\phi q_\phi(z) f(z) $$
$$ = \mathrm{E}_{z \sim q_\phi(z))}\frac{\nabla_\phi q_\phi(z)}{q_\phi(z)} f(z) $$

$$ = \mathrm{E}_{z \sim q_\phi(z)} f(z) \nabla_\phi \log q_\phi(z) $$
$$ \approx \frac1K\sum_i^K f(z^{(i)}) \nabla_\phi \log q_\phi(z^{(i)}) $$
Where $z{(i)} \sim q_\phi(z)$

This works but is very noisy.
## Reparametrization trick

In the case of specific distributions, we may be able to do better. 

E.g. in the case of a gaussian, $q_\phi(z) = \mathcal{N}(\mu, \sigma^2)$. Then we can write 

$$ z = \mu + \epsilon * \sigma \quad \epsilon \sim \mathcal{N}(0, 1)$$

$$ \mathrm{E}_{z \sim q_\phi(z)} [f(z)] = \mathrm{E}_{z \sim q_\phi(z)} [f(\mu + \epsilon \sigma)] \approx \frac1K\sum_i^K f(\mu + \epsilon\sigma)$$
Then the gradient is 

$$ \nabla_{\mu,\sigma} \left(\mathrm{E}_{z \sim q_\phi(z)}[f(z)]\right) \approx 
\nabla_{\mu,\sigma} \frac1K\sum_i^K f(\mu + \epsilon\sigma) 
= \frac1K\sum_i^K \nabla_{\mu, \sigma} f(\mu + \epsilon^{(i)}\sigma) 
$$
This has **much lower variance** than the generic likelihood ratio gradient.
## In general

* If the variables are continuous, we can use a flow to reparametrize/approximate the distribution that we're trying to sample from, and then we can optimize over it the way we did above.

* Unsure what happens when the distribution is discrete. 
# Optimizing VLB with Likelihood Ratio Gradient


$$\max_{\theta, \phi} \mathop{\mathrm{E}}_{z \sim q_\phi(z | x^{(i)})} \left[ \log(p_z(z)) + \log(p_\theta(x^{(i)} | z)) - \log(q_\phi(z | x^{(i)}))\right]$$
Gradient with respect to $\theta$ is easy, we can just sample:

$$= \nabla_\theta \mathop{\mathrm{E}}_{z \sim q_\phi(z | x^{(i)})} 
\left[ \log p_\theta(x^{(i)} | z)\right]
\approx \nabla_\theta \frac1K\sum_{k=1}^K \log p_\theta(x^{(i)} | z^{(k)})\quad\quad z^{(k)} \sim q_\phi(z | x^{(i)})
$$
Gradient with respect to $\phi$:


$$ 
\nabla_\phi \mathop{\mathrm{E}}_{z \sim q_\phi(z | x^{(i)})} \left[ 
\log(p_z(z)) + \log(p_\theta(x^{(i)} | z)) - \log(q_\phi(z | x^{(i)}))
\right]  
$$
Expand the expectation:
$$ \approx \nabla_\phi \sum_z q_\phi(z | x^{(i)})\left[  
\log(p_z(z) + \log(p_\theta(x^{(i)} | z) - \log(q_\phi(z| x^{(i)}))
\right]
$$
$$ 
= \sum_z \nabla_\phi q_\phi(z | x^{(i)})\left[  
\log(p_z(z) + \log(p_\theta(x^{(i)} | z) - \log(q_\phi(z | x^{(i)}))
\right] - q_\phi(z | x^{(i)})\nabla_\phi \log(q_\phi(z | x^{(i)}))
$$
$$
= \sum_z \nabla_\phi q_\phi(z)\left[  
\log(p_z(z) + \log(p_\theta(x^{(i)} | z) - \log(q_\phi(z | x^{(i)}))
\right] - q_\phi(z | x^{(i)})\frac{\nabla_\phi q_\phi(z | x^{(i)})}{q_\phi(z | x^{(i)})}
$$
$$
= \sum_z \nabla_\phi q_\phi(z\left[  
\log(p_z(z) + \log(p_\theta(x^{(i)} | z) - \log(q_\phi(z | x^{(i)}))
\right] - \nabla_\phi q_\phi(z | x^{(i)})
$$
$$
= \sum_z \nabla_\phi q_\phi(z\left[  
\log(p_z(z) + \log(p_\theta(x^{(i)} | z) - \log(q_\phi(z | x^{(i)}))
\right] - \nabla_\phi \sum_z q_\phi(z | x^{(i)})
$$
And we know that for $z \sim q_\phi(z|x^{(i)})$, $\sum_z q_\phi(z | x^{(i)}) = 1$, and thus $\nabla_\phi\sum_z q_\phi(z | x^{(i)}) = 0$. Hence:
$$
= \sum_z \nabla_\phi q_\phi(z\left[  
\log(p_z(z) + \log(p_\theta(x^{(i)} | z) - \log(q_\phi(z | x^{(i)}))
\right]
$$
So we don't have to worry about the extra term.
# Questions

#### How does the [Likelihood Ratio Gradient Toy problem work](https://www.youtube.com/watch?v=FMuvUZXMzKM&list=PLwRJQ4m4UJjPiJP3691u-qWwPGVKzSlNP&index=4)? (the lecture has an error)

<img src="https://www.evernote.com/l/Agsa9w8BCmRHOL2FCOrGW9dWKmln5evBNEwB/image.png" alt="drawing" width="500"/>

$$ \approx \frac1K\sum_i^K f(z^{(i)}) \left(\frac12 (\phi - z) \right) $$
Therefore each term is moving in the direction $z \rightarrow \phi$ proportional to $f(z^{(i)})$. 

* Samples further away from the green point will contribute terms that point ~towards the green point, with a big coefficient
* Samples closer to the green point will contribute terms that point away from the green point with small coefficients. 
* The average of all of the terms (when K is big) points to the green point.
* This process is fairly noisy.
#### [How does VAE and likelihood ratio gradient work?](https://youtu.be/FMuvUZXMzKM?list=PLwRJQ4m4UJjPiJP3691u-qWwPGVKzSlNP&t=5500)
  * Why do we get an "additional term"?: Just expand the expectation and apply the gradient there.
#### [How is the objective of the VAE derived?](https://youtu.be/FMuvUZXMzKM?list=PLwRJQ4m4UJjPiJP3691u-qWwPGVKzSlNP&t=6338)

<img src="https://www.evernote.com/l/AgsRPHxedUdJjJ9jzNfbDILqMFnsqXmgZuMB/image.png" alt="drawing" width="500"/>


#### Go over VQ-VAE
* [How does it actually work?](https://arxiv.org/pdf/1711.00937.pdf)
